In [ ]:
import numpy as np
import tensorflow as tf
import random as python_random
# fix random seed for reproducibility
np.random.seed(73)
tf.random.set_seed(73)
python_random.seed(73)

In [ ]:
# Import dataset
from scipy.io import loadmat
dataset = loadmat('/content/drive/My Drive/Colab Notebooks/dataset/dataset_lab_276_dl.mat')

In [ ]:
# Extract and reshape CSI values
csi = dataset['csid_lab']
csi_abs = np.abs(csi)
csi_ang = np.angle(csi)
csi_tensor = np.concatenate((csi_abs,csi_ang),1)
csi_tensor = np.swapaxes(csi_tensor,0,3)
csi_tensor = np.swapaxes(csi_tensor,1,3)
csi_tensor = np.swapaxes(csi_tensor,2,3)
csi_tensor /= np.max(csi_tensor)

In [ ]:
# Encode labels
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
label = dataset['label_lab']
encoder = LabelEncoder()
encoder.fit(label.ravel())
encoded_Y = encoder.transform(label.ravel())
dummy_y = to_categorical(encoded_Y)

In [ ]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     csi_tensor, dummy_y, test_size=0.2, random_state=73)
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)

In [ ]:
# Hyperparameters
from tensorflow.keras import optimizers, regularizers
numKernel = 16
kernelSize = (5,5)
poolSize = (3,3)
optimizer = optimizers.SGD()
epochs = 10
batchSize = 10
numClasses = 276

In [ ]:
# Create model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense
from tensorflow.keras.layers import AveragePooling2D, BatchNormalization, Dropout, MaxPool2D
def baseline_model():
    model = Sequential()
    model.add(Conv2D(numKernel, kernelSize,
                     input_shape=csi_tensor.shape[1:]
                     ))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(AveragePooling2D(pool_size=poolSize))
    model.add(Flatten())
    model.add(Dense(numClasses))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer= optimizer,
                  metrics=['accuracy'])
    return model
model = baseline_model()

In [ ]:
# Build the model and training
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
estimator = KerasClassifier(build_fn=baseline_model, epochs=epochs,batch_size= batchSize, verbose=1)

history = estimator.fit(X_train, y_train, 
                        validation_data = (X_test,y_test))

In [ ]:
# 5-fold cross validation
from sklearn.model_selection import KFold, cross_val_score
kfold = KFold(n_splits=5, shuffle= True, random_state= 73)
crossval = cross_val_score(estimator, csi_tensor, dummy_y, cv = kfold)
print(crossval.mean())

In [ ]:
# List all data in history
import matplotlib.pyplot as plt
print(history.history.keys())
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
# Summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# Summarize history for loss
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()